In [ ]:
%pip install selenium tqdm bs4 playwright

In [ ]:
%pip install pandas

In [ ]:
import math
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

In [ ]:
# Constants
BASE_URL = (
    "https://www.footballtransfers.com/en/values/players/most-valuable-players/{}"
)
MAX_RETRIES = 8
THREADS = 7
MAX_PAGES = 1353
TIMEOUT_SECONDS = 1
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
DATA_FILE = "most_valuable_players_fast.csv"
SCRAPED_PAGES_LOG = "scraped_pages.txt"
FAILED_PAGES_LOG = "failed_pages.txt"


# Page Logging Helpers
def load_page_log(file_path):
    try:
        with open(file_path, "r") as f:
            return set(int(line.strip()) for line in f if line.strip().isdigit())
    except FileNotFoundError:
        return set()


def log_page(file_path, page_number):
    with open(file_path, "a") as f:
        f.write(f"{page_number}\n")


# Setup WebDriver
def create_driver():
    options = Options()
    options.binary_location = CHROMIUM_PATH
    options.add_argument("--headless=chrome")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.managed_default_content_settings.stylesheets": 2,
        "profile.managed_default_content_settings.cookies": 2,
        "profile.managed_default_content_settings.javascript": 1,
        "profile.managed_default_content_settings.plugins": 2,
        "profile.managed_default_content_settings.popups": 2,
        "profile.managed_default_content_settings.geolocation": 2,
        "profile.managed_default_content_settings.media_stream": 2,
    }
    options.add_experimental_option("prefs", prefs)
    return webdriver.Chrome(options=options)


# Parse HTML Page
def parse_html(page_source):
    soup = BeautifulSoup(page_source, "html.parser")
    rows = soup.find_all("tr")
    data = []

    for row in rows:
        try:
            name_tag = row.select_one("td.td-player a[title]")
            if not name_tag:
                continue
            name = name_tag.get_text(strip=True)
            player_url = "https://www.footballtransfers.com" + name_tag["href"]

            age_tag = row.select_one("td.age")
            age = age_tag.get_text(strip=True) if age_tag else None

            club_tag = row.select_one("td.td-player .sub-text a[title]")
            club = club_tag.get_text(strip=True) if club_tag else None
            club_url = (
                "https://www.footballtransfers.com" + club_tag["href"]
                if club_tag
                else None
            )

            position_tag = row.select_one("td.td-player span.sub-text")
            position = (
                position_tag.get_text(strip=True).split("•")[-1].strip()
                if position_tag
                else None
            )

            nationality_tag = row.select_one("td.td-player figure img")
            nationality = nationality_tag["alt"] if nationality_tag else None

            skill_tag = row.select_one("div.table-skill__skill")
            skill = float(skill_tag.get_text(strip=True)) if skill_tag else None

            potential_tag = row.select_one("div.table-skill__pot")
            potential = (
                float(potential_tag.get_text(strip=True)) if potential_tag else None
            )

            value_tag = row.select_one("span.player-tag")
            value = (
                value_tag.get_text(strip=True).replace("€", "") if value_tag else None
            )

            if value and "M" in value:
                market_value = float(value.replace("M", "")) * 1e6
            elif value and "K" in value:
                market_value = float(value.replace("K", "")) * 1e3
            else:
                market_value = None

            data.append(
                {
                    "Name": name,
                    "Player URL": player_url,
                    "Age": age,
                    "Club": club,
                    "Club URL": club_url,
                    "Position": position,
                    "Nationality": nationality,
                    "Skill": skill,
                    "Potential": potential,
                    "Market Value (€)": market_value,
                }
            )
        except Exception as e:
            print(f"Parse error: {e}")
    return data


# Scrape Range
def scrape_page_range(start_page, end_page, scraped_pages):
    driver = create_driver()
    wait = WebDriverWait(driver, TIMEOUT_SECONDS)

    for page in tqdm(
        range(start_page, end_page + 1),
        desc=f"Thread {start_page}-{end_page}",
        leave=False,
    ):
        if page in scraped_pages:
            print(f"Skipping page {page} (already scraped)")
            continue

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                url = BASE_URL.format(page)
                driver.get(url)
                wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "td.td-player a[title]")
                    )
                )
                data = parse_html(driver.page_source)

                if data:
                    df = pd.DataFrame(data)
                    df.to_csv(
                        DATA_FILE,
                        mode="a",
                        header=not os.path.exists(DATA_FILE),
                        index=False,
                    )
                    log_page(SCRAPED_PAGES_LOG, page)
                    print(f"Saved page {page} with {len(data)} players")
                else:
                    print(f"No data found on page {page}")
                break
            except Exception as e:
                print(f"Retry {attempt} failed on page {page}: {e}")
                time.sleep(2 * attempt)
                if attempt == MAX_RETRIES:
                    log_page(FAILED_PAGES_LOG, page)
                    print(f"Failed page {page} after {MAX_RETRIES} retries")
    driver.quit()


# Main Execution
def main():
    chunk_size = math.ceil(MAX_PAGES / THREADS)
    ranges = [
        (i, min(i + chunk_size - 1, MAX_PAGES))
        for i in range(1, MAX_PAGES + 1, chunk_size)
    ]
    scraped_pages = load_page_log(SCRAPED_PAGES_LOG)

    with ThreadPoolExecutor(max_workers=THREADS) as executor:
        futures = [
            executor.submit(scrape_page_range, start, end, scraped_pages)
            for start, end in ranges
        ]
        for future in tqdm(
            as_completed(futures), total=len(futures), desc="Scraping Progress"
        ):
            try:
                future.result()
            except Exception as e:
                print(f"Thread error: {e}")

    # Summary Report
    scraped = load_page_log(SCRAPED_PAGES_LOG)
    failed = load_page_log(FAILED_PAGES_LOG)
    all_attempted = scraped | failed
    skipped = set(range(1, MAX_PAGES + 1)) - all_attempted

    print("Summary:")
    print(f"Scraped pages: {len(scraped)}")
    print(f"Failed pages: {len(failed)} (see '{FAILED_PAGES_LOG}')")
    print(f"Skipped pages: {len(skipped)} (not attempted yet)")


if __name__ == "__main__":
    main()

In [ ]:
from io import StringIO

# Read file as raw text
with open("most_valuable_players_fast.csv", "r", encoding="utf-8") as f:
    csv_data = f.read()

# Replace doubled double-quotes (""M, AM (R)"") with proper quoting ("M, AM (R)")
csv_data = csv_data.replace('""', '"')

# Load CSV from string safely
df = pd.read_csv(StringIO(csv_data), quotechar='"', skipinitialspace=True)

# Convert numeric columns
for col in ["Age", "Rating", "Potential", "Value"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Show preview
print(df.head())

In [ ]:
from io import StringIO

import pandas as pd

# Your raw CSV string
csv_data = """Name,Player link,Age,Team,Team Link,Positions,Nationality,Rating,Potential,Value
Erling Haaland,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/erling-braut-haaland,24,Man City,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/uk/man-city,F (C),Norway,91.7,99.3,184200000.0
Lamine Yamal,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/lamine-yamal,17,Barcelona,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/es/barcelona,"M, AM (R)",Spain,83.9,100.0,183200000.0
Kylian Mbappé,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/kylian-mbappe,26,Real Madrid,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/es/real-madrid,F (C),France,89.7,93.8,153200000.0
Pedri,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/pedri-1,22,Barcelona,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/es/barcelona,"M, DM, AM (C)",Spain,83.8,97.5,137900000.0
Jude Bellingham,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/jude-bellingham,22,Real Madrid,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/es/real-madrid,"AM (C), M (CL)",England,83.1,97.7,137800000.0
Florian Wirtz,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/florian-wirtz,22,Liverpool,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/uk/liverpool,AM (C),Germany,86.3,98.8,135000000.0
Pau Cubarsí,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/pau-cubarsi,18,Barcelona,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/es/barcelona,D (C),Spain,71.7,100.0,130900000.0
Jamal Musiala,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/players/jamal-musiala,22,Bayern,https://www.footballtransfers.comhttps://www.footballtransfers.com/en/teams/de/bayern,M (C),Germany,89.4,100.0,129300000.00000001
"""

# Parse the CSV string safely
df = pd.read_csv(StringIO(csv_data), quotechar='"')

# Optional: convert numeric fields
for col in ["Age", "Rating", "Potential", "Value"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Preview clean data
print(df.head())


In [ ]:
with open("most_valuable_players_fast.csv", "r", encoding="utf-8") as f:
    csv_data = f.read()

print(csv_data[:1000])  # Preview first 500 characters

In [ ]:
import csv

import pandas as pd

fixed_rows = []
expected_columns = 10

with open("most_valuable_players_fast.csv", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()

        # If entire line is wrapped in quotes, unwrap it first
        if line.startswith('"') and line.endswith('"'):
            line = line[1:-1]

        # Fix escaped quotes like ""M, AM (R)"" to "M, AM (R)"
        line = line.replace('""', '"')

        # Parse line using csv.reader for proper quote handling
        parsed = next(csv.reader([line], quotechar='"', skipinitialspace=True))

        # If the row looks good, keep it
        if len(parsed) == expected_columns:
            fixed_rows.append(parsed)
        elif len(parsed) > expected_columns:
            # Try merging overflow into Positions field
            repaired = parsed[:5] + [", ".join(parsed[5:-4])] + parsed[-4:]
            if len(repaired) == expected_columns:
                fixed_rows.append(repaired)
            else:
                print(f"Could not fix row: {parsed}")

# Load into DataFrame
columns = [
    "Name",
    "Player URL",
    "Age",
    "Team",
    "Team Link",
    "Positions",
    "Nationality",
    "Rating",
    "Potential",
    "Value",
]
df = pd.DataFrame(fixed_rows, columns=columns)

# Convert numeric columns
for col in ["Age", "Rating", "Potential", "Value"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Preview cleaned data
df.head()

In [ ]:
df = df.drop(index=0).reset_index(drop=True)
# Define the redundant prefix
prefix = "https://www.footballtransfers.com"

# Fix repeated prefix in all relevant URL columns
for col in ["Player URL", "Team Link"]:
    if col in df.columns:
        df[col] = df[col].str.replace(f"{prefix}{prefix}", prefix, regex=False)


In [ ]:
df.to_csv("all_players_ratings.csv")

In [ ]:
# Sort by 'Value' descending
df_sorted = df.sort_values(by="Value", ascending=False)

# Preview top 10 most valuable players
df_sorted[["Name", "Team", "Value"]][:50]

In [ ]:
# Filter for French players
french_players = df[df["Nationality"] == "France"]

# Optional: sort by Value descending
french_players_sorted = french_players.sort_values(by="Rating", ascending=False)

# Show relevant columns
french_players_sorted[["Name", "Team", "Value", "Rating", "Potential"]][:50]

In [ ]:
%pip install nest_asyncio

In [ ]:
from playwright.sync_api import sync_playwright


def run_scrape():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        page.goto(
            "https://fbref.com/en/players/7a2e46a8/all_comps/Alisson-Stats---All-Competitions"
        )
        page.wait_for_selector("#stats_keeper_expanded")
        table_html = page.locator("#stats_keeper_expanded").inner_html()
        print(table_html)
        browser.close()


if __name__ == "__main__":
    run_scrape()

In [ ]:
import asyncio

import pandas as pd

CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)


async def fetch_table_and_save_csv():
    url = "https://fbref.com/en/players/7a2e46a8/all_comps/Alisson-Stats---All-Competitions"

    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto(url)

        # Wait for rows to ensure content is loaded
        await page.wait_for_selector("#stats_keeper_expanded >> tbody >> tr")
        await asyncio.sleep(2)

        html = await page.content()
        await browser.close()

    # Parse with BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", {"id": "stats_keeper_expanded"})
    if not table:
        print("Table not found.")
        return

    # Use last <tr> in <thead> as the real header row
    header_row = table.find("thead").find_all("tr")[-1]
    headers = [th.get_text(strip=True) for th in header_row.find_all("th")]

    data = []
    for tr in table.find("tbody").find_all("tr"):
        # Include potential <th> (for sticky first column like "Season")
        ths = [th.get_text(strip=True) for th in tr.find_all("th")]
        tds = [td.get_text(strip=True) for td in tr.find_all("td")]
        row = ths + tds  # combine <th> and <td> values

        if len(row) == len(headers):
            data.append(row)
        else:
            print(
                f"Row mismatch: expected {len(headers)} but got {len(row)} → Skipped row:",
                row,
            )

    if not data:
        print("No valid data rows found.")
        return

    df = pd.DataFrame(data, columns=headers)
    df.to_csv("alisson_goalkeeping_stats.csv", index=False)
    print("Saved as alisson_goalkeeping_stats.csv")


# Run it
asyncio.run(fetch_table_and_save_csv())

In [ ]:
import asyncio

import pandas as pd

CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
BASE_URL = "https://fbref.com"


async def fetch_table_and_save_csv():
    url = f"{BASE_URL}/en/players/7a2e46a8/all_comps/Alisson-Stats---All-Competitions"

    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto(url)

        # Wait for actual data rows to be visible
        await page.wait_for_selector("#stats_keeper_expanded >> tbody >> tr")
        await asyncio.sleep(2)

        html = await page.content()
        await browser.close()

    # Parse HTML content
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", {"id": "stats_keeper_expanded"})
    if not table:
        print("Table not found.")
        return

    # Get accurate column names from last header row
    header_row = table.find("thead").find_all("tr")[-1]
    headers = [th.get_text(strip=True) for th in header_row.find_all("th")]

    data = []
    for tr in table.find("tbody").find_all("tr"):
        row = []

        # Handle sticky <th> cells (e.g. "Season") and add link if present
        for th in tr.find_all("th"):
            text = th.get_text(strip=True)
            link = th.find("a")
            if link and link.get("href"):
                text += f" ({BASE_URL}{link.get('href')})"
            row.append(text)

        # Handle <td> cells with potential links
        for td in tr.find_all("td"):
            text = td.get_text(strip=True)
            link = td.find("a")
            if link and link.get("href"):
                text += f" ({BASE_URL}{link.get('href')})"
            row.append(text)

        if len(row) == len(headers):
            data.append(row)
        else:
            print(
                f"Row mismatch: expected {len(headers)} but got {len(row)} → Skipping."
            )

    if not data:
        print("No valid data rows found.")
        return

    df = pd.DataFrame(data, columns=headers)
    df.to_csv("alisson_goalkeeping_stats_upgraded.csv", index=False)
    print("Saved as alisson_goalkeeping_stats_upgraded.csv with links included")


# Execute the async function
asyncio.run(fetch_table_and_save_csv())

In [ ]:
import asyncio

import nest_asyncio
import pandas as pd
from playwright.async_api import async_playwright

# Patch event loop (required for Jupyter or interactive mode)
nest_asyncio.apply()

# Constants
BASE_URL = "https://fbref.com"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)


async def fetch_national_team_stats():
    url = f"{BASE_URL}/en/players/7a2e46a8/all_comps/Alisson-Stats---All-Competitions"

    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        page = await browser.new_page()
        await page.goto(url)

        # OPTIONAL: Wait for a key element (e.g. "Matches" tab) to ensure page loads
        await asyncio.sleep(4)  # allow time for JS-rendered tables

        # Get page HTML
        html = await page.content()
        await browser.close()

    # Parse with BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Locate the correct national team keeper table by ID
    table = soup.find("table", {"id": "stats_keeper_nat_tm"})
    if not table:
        print("National Team goalkeeping table not found.")
        return

    # Extract column headers
    header_row = table.find("thead").find_all("tr")[-1]
    headers = [th.get_text(strip=True) for th in header_row.find_all("th")]

    # Extract table rows
    data = []
    for tr in table.find("tbody").find_all("tr"):
        row = []

        # <th> cells (e.g. season) + optional link
        for th in tr.find_all("th"):
            text = th.get_text(strip=True)
            link = th.find("a")
            if link and link.get("href"):
                text += f" ({BASE_URL}{link.get('href')})"
            row.append(text)

        # <td> cells + optional link
        for td in tr.find_all("td"):
            text = td.get_text(strip=True)
            link = td.find("a")
            if link and link.get("href"):
                text += f" ({BASE_URL}{link.get('href')})"
            row.append(text)

        # Verify row matches header length
        if len(row) == len(headers):
            data.append(row)
        else:
            print(f"Skipped row ({len(row)} vs {len(headers)} columns):", row)

    # Export to CSV
    if data:
        df = pd.DataFrame(data, columns=headers)
        df.to_csv("alisson_national_team_stats.csv", index=False)
        print("Saved as alisson_national_team_stats.csv with embedded links")
    else:
        print("No valid rows found.")


# Run the function in Notebook
await fetch_national_team_stats()

In [ ]:
import asyncio

import nest_asyncio
import pandas as pd

nest_asyncio.apply()

# Settings
BASE_URL = "https://fbref.com"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
SAVE_FOLDER = "alisson_all_fbref_tables"

os.makedirs(SAVE_FOLDER, exist_ok=True)


async def scrape_all_fbref_tables():
    url = f"{BASE_URL}/en/players/7a2e46a8/all_comps/Alisson-Stats---All-Competitions"

    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        page = await browser.new_page()
        await page.goto(url)

        # Trigger all preset buttons to reveal toggled tables
        await page.evaluate("""
            document.querySelectorAll('a.sr_preset').forEach(el => el.click());
        """)
        await asyncio.sleep(5)  # Let the page render updated sections

        html = await page.content()
        await browser.close()

    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")

    print(f"🔍 Found {len(tables)} tables on page")

    for table in tables:
        table_id = table.get("id", None)
        if not table_id:
            continue  # skip tables without ID

        print(f"Processing table: {table_id}")

        # Extract headers
        header_row = (
            table.find("thead").find_all("tr")[-1] if table.find("thead") else None
        )
        headers = (
            [th.get_text(strip=True) for th in header_row.find_all("th")]
            if header_row
            else []
        )

        rows = []
        for tr in table.find("tbody").find_all("tr"):
            row = []
            for cell in tr.find_all(["th", "td"]):
                text = cell.get_text(strip=True)
                link = cell.find("a")
                if link and link.get("href"):
                    text += f" ({BASE_URL}{link.get('href')})"
                row.append(text)
            if len(row) == len(headers):
                rows.append(row)

        if rows:
            df = pd.DataFrame(rows, columns=headers)
            output_path = os.path.join(SAVE_FOLDER, f"{table_id}.csv")
            df.to_csv(output_path, index=False)
            print(f"Saved: {output_path}")
        else:
            print(f"Skipped empty table: {table_id}")


# Run the function
await scrape_all_fbref_tables()

In [ ]:
import pandas as pd

# Replace with your actual file path or name
df = pd.read_csv("all_players_ratings.csv")

# Optional: Preview the first few rows
print(df.head())

In [ ]:
import asyncio

import nest_asyncio
import pandas as pd

nest_asyncio.apply()

CSV_PATH = "all_players_ratings.csv"  # Update with your actual file
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
SEARCH_BASE = "https://fbref.com/en/search/search.fcgi?search="

# Load player list
df = pd.read_csv(CSV_PATH)
df["fbref_url"] = None  # New column to store FBref profile link


async def search_fbref_for_players():
    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        page = await browser.new_page()

        for i, row in df.iterrows():
            name = str(row["Name"]).strip()
            search_url = SEARCH_BASE + name.replace(" ", "+")
            await page.goto(search_url)
            await page.wait_for_timeout(1500)  # small delay to allow page content

            # Look for first profile link
            links = await page.locator("a").all()
            match = None
            for link in links:
                href = await link.get_attribute("href")
                if href and "/en/players/" in href and "all_comps" in href:
                    match = "https://fbref.com" + href
                    break

            if match:
                df.at[i, "fbref_url"] = match
                print(f"Found: {name} → {match}")
            else:
                print(f"Not found: {name}")

        await browser.close()

    # Export enriched data
    output_path = "players_with_fbref_links.csv"
    df.to_csv(output_path, index=False)
    print(f"All done! Saved to {output_path}")


# Run the function
await search_fbref_for_players()

In [ ]:
import asyncio

import nest_asyncio
import pandas as pd

nest_asyncio.apply()

CSV_PATH = "all_players_ratings.csv"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
SEARCH_BASE = "https://fbref.com/en/search/search.fcgi?search="

df = pd.read_csv(CSV_PATH)
df["fbref_url"] = None


async def search_fbref_for_players():
    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        page = await browser.new_page()

        for i, row in df.iterrows():
            name = str(row["Name"]).strip()
            search_url = SEARCH_BASE + name.replace(" ", "+")
            await page.goto(search_url)
            await page.wait_for_timeout(1500)

            links = await page.locator("a").all()
            match_url = None

            for link in links:
                href = await link.get_attribute("href")
                if href and "/en/players/" in href:
                    try:
                        after_players = href.split("/players/")[1]
                        player_id = after_players.split("/")[0]
                        name_slug = name.replace(" ", "-") + "-Stats---All-Competitions"
                        match_url = f"https://fbref.com/en/players/{player_id}/all_comps/{name_slug}"
                        break
                    except Exception:
                        continue

            if match_url:
                df.at[i, "fbref_url"] = match_url
                print(f"{name} → {match_url}")
            else:
                print(f"Not found: {name}")

        await browser.close()
        df.to_csv("players_with_fbref_links.csv", index=False)
        print("Saved as players_with_fbref_links.csv")


await search_fbref_for_players()

In [ ]:
import asyncio

import nest_asyncio
import pandas as pd

nest_asyncio.apply()

# Configuration
CSV_PATH = "all_players_ratings.csv"  # Update with your actual file path
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
DUCKDUCKGO_SEARCH = "https://duckduckgo.com/?q=site%3Afbref.com+"

# Load players from CSV
df = pd.read_csv(CSV_PATH)
df["fbref_url"] = None  # Add column for results


async def search_fbref_via_duckduckgo():
    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=False)
        page = await browser.new_page()

        # Loop through players
        for i, row in df.iterrows():
            name = str(row["Name"]).strip()
            query_url = DUCKDUCKGO_SEARCH + name.replace(" ", "+")
            await page.goto(query_url)
            await page.wait_for_timeout(2000)  # Wait for search results to load

            match = None
            links = await page.locator("a").all()

            for link in links:
                href = await link.get_attribute("href")
                if href and "fbref.com/en/players/" in href:
                    match = href
                    break

            if match:
                df.at[i, "fbref_url"] = match
                print(f"Found: {name} → {match}")
            else:
                print(f"Not found: {name}")

        # Clean up
        await browser.close()

    # Save results
    output_path = "players_with_fbref_links_duckduckgo.csv"
    df.to_csv(output_path, index=False)
    print(f"All done! Saved to {output_path}")


# Run the scraping function
await search_fbref_via_duckduckgo()

In [ ]:
import asyncio
from datetime import datetime

import nest_asyncio
import pandas as pd
from tqdm.asyncio import tqdm

nest_asyncio.apply()

CSV_PATH = "all_players_ratings.csv"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
DUCKDUCKGO_SEARCH = "https://duckduckgo.com/?q=site%3Afbref.com+"
LOG_PATH = "scraping_log.txt"
NUM_TABS = 5
MAX_RETRIES = 3

df = pd.read_csv(CSV_PATH)
df["fbref_url"] = df.get("fbref_url", None)

with open(LOG_PATH, "w", encoding="utf-8") as log:
    log.write(f"Scraping started at {datetime.now()}\n\n")


# Scraping logic with retries
async def scrape_player(tab, index, name):
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            query_url = DUCKDUCKGO_SEARCH + name.replace(" ", "+")
            await tab.goto(query_url)
            await tab.wait_for_timeout(1500)

            links = await tab.locator("a").all()
            for link in links:
                href = await link.get_attribute("href")
                if href and "fbref.com/en/players/" in href:
                    df.at[index, "fbref_url"] = href
                    with open(LOG_PATH, "a", encoding="utf-8") as log:
                        log.write(f"Found (Attempt {attempt}): {name} → {href}\n")
                    return
            # If we get here, no match found
        except Exception as e:
            with open(LOG_PATH, "a", encoding="utf-8") as log:
                log.write(f" Error on {name} (Attempt {attempt}): {e}\n")

    # Final failure
    with open(LOG_PATH, "a", encoding="utf-8") as log:
        log.write(f"Not found after {MAX_RETRIES} attempts: {name}\n")


async def scrape_batch(tab, batch):
    for index, name in tqdm(
        batch, desc=f"Tab scraping {len(batch)} players", leave=False
    ):
        await scrape_player(tab, index, name)


async def search_fbref_multi_tab():
    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=False)
        tabs = [await browser.new_page() for _ in range(NUM_TABS)]

        all_players = [(i, str(row["Name"]).strip()) for i, row in df.iterrows()]
        batch_size = math.ceil(len(all_players) / NUM_TABS)
        batches = [
            all_players[i : i + batch_size]
            for i in range(0, len(all_players), batch_size)
        ]

        await asyncio.gather(
            *[scrape_batch(tab, batch) for tab, batch in zip(tabs, batches)]
        )

        await browser.close()

    df.to_csv(CSV_PATH, index=False)
    print("Scraping finished with retries and progress bar. CSV and log updated.")


await search_fbref_multi_tab()

In [ ]:
import asyncio
import os
from datetime import datetime

import nest_asyncio
import pandas as pd
from tqdm import tqdm

nest_asyncio.apply()

# Configuration
CSV_PATH = "all_players_ratings.csv"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
FBREF_SEARCH = "https://fbref.com/en/search/search.fcgi?search="
LOG_PATH = "scraping_log.txt"
NUM_TABS = 1
MAX_RETRIES = 3
REQUEST_THROTTLE_SEC = 2
STARTUP_DELAY_SEC = 0

# Load CSV
df = pd.read_csv(CSV_PATH)
df["fbref_url"] = df.get("fbref_url", None)

# Init log file
with open(LOG_PATH, "w", encoding="utf-8") as log:
    log.write(f"Scraping initialized at {datetime.now()}\n\n")


# Player scraping logic
async def scrape_player(tab, index, name):
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            search_url = FBREF_SEARCH + name.replace(" ", "+")
            await tab.goto(search_url)
            await tab.wait_for_timeout(1500)
            await asyncio.sleep(REQUEST_THROTTLE_SEC)

            match = None
            links = await tab.locator("a").all()
            for link in links:
                href = await link.get_attribute("href")
                if href and "/en/players/" in href:
                    match = "https://fbref.com" + href
                    break

            if not match:
                table_links = await tab.locator("table a").all()
                for link in table_links:
                    href = await link.get_attribute("href")
                    if href and "/en/players/" in href:
                        match = "https://fbref.com" + href
                        break

            with open(LOG_PATH, "a", encoding="utf-8") as log:
                if match:
                    df.at[index, "fbref_url"] = match
                    log.write(f"Found (Attempt {attempt}): {name} → {match}\n")
                    return
                else:
                    log.write(f"No match on attempt {attempt}: {name}\n")

        except Exception as e:
            with open(LOG_PATH, "a", encoding="utf-8") as log:
                log.write(f"Error on {name} (Attempt {attempt}): {e}\n")

    with open(LOG_PATH, "a", encoding="utf-8") as log:
        log.write(f"Not found after {MAX_RETRIES} attempts: {name}\n")


# 🧹 Batch logic
async def scrape_batch(tab, batch):
    for index, name in tqdm(
        batch, desc=f" Tab scraping {len(batch)} players", leave=False
    ):
        await scrape_player(tab, index, name)


# Orchestration
async def search_fbref_multi_tab():
    print("Waiting 1 hour before scraping...")
    await asyncio.sleep(STARTUP_DELAY_SEC)
    print("Starting scraping...")

    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=False)
        tabs = [await browser.new_page() for _ in range(NUM_TABS)]

        all_players = [(i, str(row["Name"]).strip()) for i, row in df.iterrows()]
        batch_size = math.ceil(len(all_players) / NUM_TABS)
        batches = [
            all_players[i : i + batch_size]
            for i in range(0, len(all_players), batch_size)
        ]

        await asyncio.gather(
            *[scrape_batch(tab, batch) for tab, batch in zip(tabs, batches)]
        )

        await browser.close()

    df.to_csv(CSV_PATH, index=False)
    print(f"Scraping finished. Updated CSV and log saved to {LOG_PATH}")


# Run it
await search_fbref_multi_tab()

In [ ]:
import asyncio
from datetime import datetime

import nest_asyncio
import pandas as pd
from tqdm.asyncio import tqdm

nest_asyncio.apply()

# Configuration
CSV_PATH = "all_players_ratings.csv"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
GOOGLE_SEARCH = "https://www.google.com/search?q=site%3Afbref.com+"
LOG_PATH = "scraping_log.txt"
NUM_TABS = 1
MAX_RETRIES = 3
REQUESTS_PER_MIN = 30
STARTUP_DELAY_SEC = 0

df = pd.read_csv(CSV_PATH)
df["fbref_url"] = df.get("fbref_url", None)

# Initialize log file
with open(LOG_PATH, "w", encoding="utf-8") as log:
    log.write(f"Scraping initialized at {datetime.now()}\n\n")

# Rate limiter across tabs
rate_lock = asyncio.Lock()
RATE_INTERVAL = 60 / REQUESTS_PER_MIN  # seconds between each request


# Scraping logic with shared global throttle
async def scrape_player(tab, index, name, club):
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            async with rate_lock:
                await asyncio.sleep(RATE_INTERVAL)  # enforce request interval globally

            query = f"{name} {club}".replace(" ", "+")
            search_url = GOOGLE_SEARCH + query
            await tab.goto(search_url)
            await tab.wait_for_timeout(2000)

            match = None
            links = await tab.locator("a").all()
            for link in links:
                href = await link.get_attribute("href")
                if href and "fbref.com/en/players/" in href:
                    match = href
                    break

            with open(LOG_PATH, "a", encoding="utf-8") as log:
                if match:
                    df.at[index, "fbref_url"] = match
                    log.write(f"Found (Attempt {attempt}): {name} ({club}) → {match}\n")
                    return
                else:
                    log.write(f"🔍 No match on attempt {attempt}: {name} ({club})\n")

        except Exception as e:
            with open(LOG_PATH, "a", encoding="utf-8") as log:
                log.write(f"Error on {name} ({club}) (Attempt {attempt}): {e}\n")

    with open(LOG_PATH, "a", encoding="utf-8") as log:
        log.write(f"Not found after {MAX_RETRIES} attempts: {name} ({club})\n")


# Batch logic
async def scrape_batch(tab, batch):
    for index, name, club in tqdm(
        batch, desc=f"Scraping {len(batch)} players", leave=False
    ):
        await scrape_player(tab, index, name, club)


# Orchestration
async def search_fbref_multi_tab():
    if STARTUP_DELAY_SEC:
        print(f"Waiting {STARTUP_DELAY_SEC} seconds before scraping...")
        await asyncio.sleep(STARTUP_DELAY_SEC)
    print("Starting scraping...")

    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=False)
        tabs = [await browser.new_page() for _ in range(NUM_TABS)]

        all_players = [
            (i, str(row["Name"]).strip(), str(row.get("Club", "")).strip())
            for i, row in df.iterrows()
        ]
        batch_size = math.ceil(len(all_players) / NUM_TABS)
        batches = [
            all_players[i : i + batch_size]
            for i in range(0, len(all_players), batch_size)
        ]

        await asyncio.gather(
            *[scrape_batch(tab, batch) for tab, batch in zip(tabs, batches)]
        )

        await browser.close()

    df.to_csv(CSV_PATH, index=False)
    print(f"Done! Updated CSV and saved logs to {LOG_PATH}")


# Launch
await search_fbref_multi_tab()

In [ ]:
import asyncio
from datetime import datetime

import nest_asyncio
import pandas as pd

nest_asyncio.apply()

# Configuration
CSV_PATH = "all_players_ratings.csv"
DUCKDUCKGO_SEARCH = "https://duckduckgo.com/?q=site%3Afbref.com+"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
LOG_PATH = "scraping_log.txt"
RATE_LIMIT = 60 * 20  # requests per minute
RATE_INTERVAL = 60 / RATE_LIMIT  # seconds between requests

# Load CSV
df = pd.read_csv(CSV_PATH)
if "fbref_url" not in df.columns:
    df["fbref_url"] = None

# Init log
with open(LOG_PATH, "w", encoding="utf-8") as log:
    log.write(f"Scraping started at {datetime.now()}\n\n")


# Single player search
async def scrape_player(tab, index, name, club):
    await asyncio.sleep(RATE_INTERVAL)  # throttle globally

    query = f"{name} {club}".replace(" ", "+")
    url = DUCKDUCKGO_SEARCH + query + "&ia=web"
    match = None

    try:
        await tab.goto(url)
        # await tab.wait_for_timeout(500)

        links = await tab.locator("a").all()
        for link in links:
            href = await link.get_attribute("href")
            if href and "fbref.com/en/players/" in href:
                match = href
                break

        with open(LOG_PATH, "a", encoding="utf-8") as log:
            if match:
                df.at[index, "fbref_url"] = match
                log.write(f"Found: {name} ({club}) → {match}\n")
            else:
                log.write(f"Not found: {name} ({club})\n")

    except Exception as e:
        with open(LOG_PATH, "a", encoding="utf-8") as log:
            log.write(f"Error on {name} ({club}): {e}\n")


# Scrape all players
async def run_scraper():
    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=False)
        tab = await browser.new_page()

        for i, row in tqdm(df.iterrows(), total=len(df), desc="Scraping FBref links"):
            if pd.notnull(row["fbref_url"]):
                continue  # already scraped
            else:
                name = str(row["Name"]).strip()
                club = str(row.get("Club", "")).strip()
                await scrape_player(tab, i, name, club)

        await browser.close()

    df.to_csv(CSV_PATH, index=False)
    print(f"Done! CSV saved to {CSV_PATH} | Logs saved to {LOG_PATH}")


# Launch
await run_scraper()

In [ ]:
import asyncio

import nest_asyncio
import pandas as pd
from playwright.async_api import TimeoutError
from tqdm import tqdm

nest_asyncio.apply()

# Configuration
CSV_PATH = "all_players_ratings.csv"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
DUCKDUCKGO_SEARCH = "https://duckduckgo.com/?q=site%3Afbref.com+"
MAX_RETRIES = 3
RETRY_DELAY = 500  # milliseconds

# Load players from CSV
df = pd.read_csv(CSV_PATH)
if "fbref_url" not in df.columns:
    df["fbref_url"] = None


async def search_fbref_via_duckduckgo():
    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=False)
        page = await browser.new_page()

        for i, row in tqdm(df.iterrows(), total=len(df), desc="🔎 Searching players"):
            if pd.notna(row["fbref_url"]):
                print(f"⏩ Skipped: {row['Name']} (already found)")
                continue

            name = str(row["Name"]).strip()
            query_url = DUCKDUCKGO_SEARCH + name.replace(" ", "+") + "&ia=web"
            match = None

            for attempt in range(1, MAX_RETRIES + 1):
                try:
                    await page.goto(query_url, wait_until="domcontentloaded")
                    await page.wait_for_timeout(400)

                    links = await page.locator("a").all()
                    for link in links:
                        href = await link.get_attribute("href")
                        if href and "fbref.com/en/players/" in href:
                            match = href
                            break

                    if match:
                        break

                except TimeoutError:
                    print(f"⚠️ Timeout on attempt {attempt} for {name}")
                    await page.wait_for_timeout(RETRY_DELAY)

            if match:
                df.at[i, "fbref_url"] = match
                print(f"Found: {name} → {match}")
                df.to_csv(CSV_PATH, index=False)  # Save immediately
            else:
                print(f"Not found after {MAX_RETRIES} attempts: {name}")

        await browser.close()

    print(f"All done! Saved to {CSV_PATH}")


# Run the scraping function
await search_fbref_via_duckduckgo()

In [ ]:
%pip install asyncio nest_asyncio

In [ ]:
%pip install playwright==1.39.0 nest_asyncio==1.5.8

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import asyncio
import platform

if platform.system() == "Windows":
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

In [ ]:
import asyncio
import random

import nest_asyncio
import pandas as pd
from tqdm.asyncio import tqdm

asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
nest_asyncio.apply()

# Configuration
CSV_PATH = "all_players_ratings.csv"
CHROMIUM_PATH = (
    r"C:/Users/L1160681/playwright-browsers/chromium-win64/chrome-win/chrome.exe"
)
DUCKDUCKGO_SEARCH = "https://duckduckgo.com/?q=site%3Afbref.com+"
MAX_RETRIES = 3
RETRY_DELAY = 0.5  # In seconds
CONCURRENT_WORKERS = 10

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64)...",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)...",
    "Mozilla/5.0 (X11; Linux x86_64)...",
]

PROXY = {
    "server": "http://your.proxy.server:port",
    "username": "proxy_user",
    "password": "proxy_pass",
}

# Load and shuffle players
df = pd.read_csv(CSV_PATH)
if "fbref_url" not in df.columns:
    df["fbref_url"] = None
df = df.sample(frac=1).reset_index(drop=True)


async def process_player(context, i, row):
    if pd.notna(row["fbref_url"]):
        return

    name = str(row["Name"]).strip()
    query_url = DUCKDUCKGO_SEARCH + name.replace(" ", "+") + "&ia=web"
    match = None
    page = await context.new_page()

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            await page.goto(query_url, wait_until="domcontentloaded", timeout=10000)
            links = await page.locator("a").all()
            for link in links:
                href = await link.get_attribute("href")
                if href and "fbref.com/en/players/" in href:
                    match = href
                    break
            if match:
                break
        except Exception as e:
            print(f"Attempt {attempt} failed for {name}: {e}")
            await asyncio.sleep(RETRY_DELAY)

    await page.close()

    if match:
        df.at[i, "fbref_url"] = match
        print(f"{name}: {match}")
    else:
        print(f"{name}: Not found")

    await asyncio.sleep(random.uniform(1.5, 4.0))


async def search_fbref_concurrent():
    async with async_playwright() as p:
        browser = await p.chromium.launch(executable_path=CHROMIUM_PATH, headless=True)
        sem = asyncio.Semaphore(CONCURRENT_WORKERS)

        async def limited_worker(i, row):
            async with sem:
                agent = random.choice(USER_AGENTS)
                context = await browser.new_context(user_agent=agent, proxy=PROXY)
                try:
                    await process_player(context, i, row)
                finally:
                    await context.close()

        tasks = [limited_worker(i, row) for i, row in df.iterrows()]
        for f in tqdm(
            asyncio.as_completed(tasks), total=len(tasks), desc="🚀 Scraping"
        ):
            await f

        await browser.close()
        df.to_csv(CSV_PATH, index=False)
        print(f"Completed. Saved to {CSV_PATH}")


# Run the concurrent scraper
asyncio.run(search_fbref_concurrent())


In [ ]:
%pip install certifi

In [ ]:
import ssl

import certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())